In [ ]:
# Do utworzenia Stats_2024.html
import requests # make a request to the webpage to download it
import time

url = "https://www.basketball-reference.com/leagues/NBA_2024_totals.html"

# create a url
data = requests.get(url)
    
# W+ opens file in write mode and if it already exists it will just overwrite.
with open("Stats_2024.html", "w+", encoding = "utf-8") as f:
    time.sleep(3)
    f.write(data.text) #text saves files as html

In [ ]:

# import beautiful soup
from bs4 import BeautifulSoup
import pandas as pd

# read the HTML data
with open("Stats_2024.html", encoding="utf-8") as f:
    page = f.read()

# create a parser class to extract table from the page
soup = BeautifulSoup(page, "html.parser")

# remove the top row of the table
'''soup.find("tr", class_="over_header").decompose()
print("Header row removed successfully")'''

# find the specific table we want using its id
mvp_table = soup.find(id="all_totals_stats")
Stats = pd.read_html(str(mvp_table))

# Data Frame 
df = pd.DataFrame(Stats[0])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

numeric_columns = ['Rk' ,'Age', 'G' , 'GS'  ,  'MP'  , 'FG'  , 'FGA' ,  'FG%'  , '3P' , '3PA' ,  '3P%' ,  '2P' , '2PA'  , '2P%' , 'eFG%',   'FT', 'FTA' ,  'FT%',  'ORB'  ,'DRB' , 'TRB',  'AST' ,'STL' ,'BLK',  'TOV'  , 'PF',   'PTS']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# usunięcie tych które mają w Rk nan (wszędzie nan)
df = df.dropna(subset='Rk')

#usunięcie powturzeń (zostają wyniki z całego sezonu)
df = df.loc[df.groupby('Player')['G'].idxmax()]

# Zastąpienie NaN przez 0
df = df.fillna(0)

string_columns = ['Player', 'Pos', 'Tm']
# Zamiana wybranych kolumn na typ string
df[string_columns] = df[string_columns].astype(str)


In [ ]:
# Średnio punktów na mecz
df['PPG'] = df['PTS'] / df['G']
# Średnio minut na mecz
df['MPG'] = df['MP'] / df['G']
df['ORBPG'] = df['ORB'] / df['G']
df['DRBPG'] = df['DRB'] / df['G']
df['TRBPG'] = df['TRB'] / df['G']
df['ASTPG'] = df['AST'] / df['G']
df['STLPG'] = df['STL'] / df['G']
df['BLKPG'] = df['BLK'] / df['G']
df['TOVPG'] = df['TOV'] / df['G']
df['PFPG'] = df['PF'] / df['G']

# Utworzenie zmiennej sprawdzającej czy ktoś jest gwiazgą ;)
df['Star'] = ((df['GS'] > ((2*df['G']) / 3)) & (df['G'] > 60)).astype(int)
count_of_stars = df['Star'].sum()

print("Liczba nowych zmiennych z wartością 1:", count_of_stars)

# Utworzenie zmiennej sprawdzającej czy ktoś jest aktywnie grający ;)
df['Active'] = ((df['MPG'] > 8) & (df['G'] > 40)).astype(int)
count_of_activ = df['Active'].sum()

'''
Bardzo ważne: Zostawiamy tylko zawodników którzy faktycznie grają
'''
df = df[df['Active'] == 1]

# Dalej już bez RK (Numer pożądkowy)
numeric_columns = ['Age',  'G', 'GS', 'eFG%', 'FT%', 'TRBPG', 'ASTPG', 'STLPG', 'BLKPG', 'TOVPG', 'PFPG', 'PPG', 'MPG']

# Procenty *100
percent_columns = ['FG%', '3P%','2P%', 'eFG%', 'FT%', ]
df[percent_columns] = df[percent_columns] * 100

df = df.loc[:, ['Age','Player', 'Pos', 'Tm', 'G', 'GS', 'eFG%', 'FT%', 'TRBPG', 'ASTPG', 'STLPG', 'BLKPG', 'TOVPG', 'PFPG', 'PPG', 'MPG', 'Star', 'Active']]

# Eksportowanie DataFrame do pliku CSV
df.to_csv('df.csv', index=True, index_label='Index')

In [ ]:
import pandas as pd
import numpy as np

# Zakładamy, że masz DataFrame o nazwie df
# Funkcja do usuwania wartości odstających na podstawie IQR
def remove_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    mask = ~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)
    return df[mask]

# Usunięcie wartości odstających
df_cleaned = remove_outliers_iqr(df.select_dtypes(include=[np.number]))

# Dodanie kolumn nie numerycznych z powrotem do df_cleaned
df = df_cleaned.join(df[df.columns.difference(df_cleaned.columns)])

# Eksportowanie DataFrame do pliku CSV
df.to_csv('df_po_wyczyszceniu_IQR.csv', index=True, index_label='Index')

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, shapiro, jarque_bera

# Sprawdzenie Normalności rozkładu df

# Zainicjalizowanie słowników do przechowywania wyników
skewness = {}
kurtosis_values = {}
shapiro_p_values = {}
jb_p_values = {}

# Iteracja po każdej kolumnie DataFrame
for column in df.select_dtypes(include=[np.number]).columns:
    # Obliczanie skośności i kurtozy
    skewness[column] = skew(df[column].dropna())
    kurtosis_values[column] = kurtosis(df[column].dropna())
    
    # Przeprowadzenie testu Shapiro-Wilka
    stat, p_value = shapiro(df[column].dropna())
    shapiro_p_values[column] = p_value

    # Przeprowadzenie testu Jarque-Bera
    jb_stat, jb_p_value = jarque_bera(df[column].dropna())
    jb_p_values[column] = jb_p_value


# Tworzenie DataFrame z wynikami
results = pd.DataFrame({
    'Skośność': skewness,
    'Kurtoza': kurtosis_values,
    'Shapiro-Wilk p-value': shapiro_p_values,
    'Jarque-Bera p-value': jb_p_values
})

# Wyświetlenie wyników
print(results)

# Interpretacja wyników Shapiro-Wilk
alpha = 0.05
results['Normalność_S-W'] = results['Shapiro-Wilk p-value'].apply(lambda p: 'Normalny' if p > alpha else 'Nie normalny')

# Interpretacja wyników Jarque-Bera
alpha = 0.05
results['Normalność_J-B'] = results['Jarque-Bera p-value'].apply(lambda p: 'Normalny' if p > alpha else 'Nie normalny')


# Eksportowanie DataFrame do pliku CSV
results.to_csv('results.csv', index=True, index_label='Index')

In [ ]:
description = df.describe()

# Eksportowanie DataFrame do pliku CSV
description.to_csv('description.csv', index=True, index_label='Index')

# tu jest numpy arr, niewiem jescze po co ale jest xD
'''Stats_arr = df.to_numpy()
print(type(Stats_arr))
print(Stats_arr[0])'''

In [ ]:
#print(df.loc[0])

print("\nŚrednia dla wybranych kolumn:")
print(df[numeric_columns].mean())  # średnia
print("\nMediana dla wybranych kolumn:")
print(df[numeric_columns].median())  # mediana
print("\nModa dla wybranych kolumn:")
print(df[numeric_columns].mode())  # moda
print("\nOdchylenie standardowe dla wybranych kolumn:")
print(df[numeric_columns].std())  # odchylenie standardowe

In [ ]:
# Tworzenie wykresu box plot dla każdej kolumny
plt.figure(figsize=(12, 20))
for i, column in enumerate(numeric_columns, 1):
    #plt.figure(figsize=(7, 4))
    plt.subplot(9, 3, i)  # Ustawiamy aktualny subplot w siatce 3x9
    sns.boxplot(y=df[column])
    plt.title(f'Box plot for {column}')
    plt.ylabel(column)
    #plt.show()

plt.tight_layout()  # Dostosowujemy układ subplotów, aby uniknąć nachodzenia się
plt.show()  # Wyświetlamy wykresy

In [ ]:
# Znalezienie wartości NaN w DataFrame
nan_values = df.isna()
print("Wartości NaN w DataFrame:")
print(nan_values)

# Znalezienie wartości równych 0 w DataFrame
zero_values = df == 0
print("Wartości równe 0 w DataFrame:")
print(zero_values)

# Wartości NaN w wybranych kolumnach
print("\nWartości NaN w wybranych kolumnach:")
print(df[numeric_columns].isna())

# Wartości równe 0 w wybranych kolumnach
print("\nWartości równe 0 w wybranych kolumnach:")
print(df[numeric_columns] == 0)

In [ ]:
# Wyświetlenie wierszy zawierających NaN w wybranych kolumnach
print("\nWiersze zawierające NaN w wybranych kolumnach:")
print(df[numeric_columns][df[numeric_columns].isna().any(axis=1)])

# Wyświetlenie wierszy zawierających wartości równe 0 w wybranych kolumnach
print("\nWiersze zawierające wartości równe 0 w wybranych kolumnach:")
print(df[numeric_columns][(df[numeric_columns] == 0).any(axis=1)])

In [ ]:
# Wyznaczenie macierzy korelacji
correlation_matrix = df[numeric_columns].corr()

# Wyświetlenie macierzy korelacji
print("Macierz korelacji:")
print(correlation_matrix)

In [ ]:
# Ustawienie rozmiaru wykresu
plt.figure(figsize=(18, 12))

# Tworzenie heatmapy macierzy korelacji
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")

# Dodanie tytułu
plt.title('Macierz korelacji')

# Wyświetlenie wykresu
plt.show()

In [ ]:
# Tworzenie wykresu
plt.figure(figsize=(12, 20))

for i, column in enumerate(numeric_columns, 1):
    plt.subplot(9, 3, i)  # Ustawiamy aktualny subplot w siatce 3x9
    df[column].hist(figsize=(12, 20), bins = 10, edgecolor='black')
    plt.title(f'Histogram for {column}')
    plt.xlabel(column)
    #plt.show()

plt.tight_layout()  # Dostosowujemy układ subplotów, aby uniknąć nachodzenia się
plt.show()  # Wyświetlamy wykresy


In [ ]:
# Przykład: Zależność
x_column = 'eFG%'
y_column = 'PPG'

# Stworzenie wykresu rozrzutu
plt.figure(figsize=(7, 4))
plt.scatter(df[x_column], df[y_column], s=5, alpha=0.5)
plt.title(f'Zależność między {x_column} a {y_column}')
plt.xlabel(x_column)
plt.ylabel(y_column)
plt.grid(False)

# Wyświetlenie wykresu
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Sprawdzenie typów danych i przekształcenie kolumny "Pos" na kategorię
df['Pos'] = df['Pos'].astype('category')

# Wybieranie cech i etykiet
X = df.loc[:, [ 'eFG%', '3P%','2P%','FT%',  'PPG', 'MPG', 'TRBPG',  'ASTPG' ,'STLPG' ,'BLKPG',  'TOVPG'  , 'PFPG']]
y = df['Pos']  # Etykiety (kolumna "Pos")

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)

# Normalizacja danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Trenowanie modelu KNN
knn = KNeighborsClassifier(n_neighbors=10)  # liczba sąsiadów
knn.fit(X_train, y_train)

# Predykcja na zbiorze testowym
y_pred = knn.predict(X_test)

# Ocena modelu
Confusion_matrix = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))
Accuracy_score = accuracy_score(y_test, y_pred)

In [ ]:
import statsmodels.api as sm
# Model liniowy

# Wybieranie cech i etykiet
X = df.loc[:, ['G','GS', 'eFG%', 'FT%', 'TRBPG', 'ASTPG', 'STLPG', 'BLKPG', 'PFPG']]
y = df['PPG']

# Dodanie kolumny stałej (intercept)
X = sm.add_constant(X)

# Zbudowanie modelu regresji liniowej
model = sm.OLS(y, X).fit()

# Wyświetlenie podsumowania modelu
print(model.summary())
# Wyodrębnienie wartości p dla każdego współczynnika
p_values = model.pvalues

In [ ]:

import statsmodels.api as sm
import numpy as np

# Przykładowe statystyki zawodnika
player_stats = {
    'const': [1],
    'G': [82],           
    'GS': [82],          
    'eFG%': [0.55],      
    'FT%': [0.85],       
    'TRBPG': [10.0],     
    'ASTPG': [5.0],      
    'STLPG': [1.5],      
    'BLKPG': [3.0],      
    'TOVPG': [2.0],      
    'PFPG': [3.0]        
}

# Tworzenie DataFrame z przykładowymi statystykami
player_df = pd.DataFrame(player_stats)

# Dodawanie stałej kolumny do DataFrame
player_df = sm.add_constant(player_df)

# Dokonywanie predykcji
predicted_pts = model.predict(player_df)

# Wyświetlanie przewidywanej liczby punktów na mecz
print(f"Przewidywana liczba punktów na mecz: {predicted_pts[0]}")

In [ ]:
y_pred = model.predict(X)

# Obliczenie reszt
residuals = y - y_pred
standardized_residuals = (residuals - np.mean(residuals)) / np.std(residuals)

# Rysowanie rzeczywistych vs przewidywanych wartości punktów
plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred, alpha=0.5, s=10)
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', lw=1)  # Linia idealna
plt.xlabel('Rzeczywiste PTS')
plt.ylabel('Przewidywane PTS')
plt.title('Rzeczywiste vs Przewidywane PTS')
plt.grid(True)
plt.show()


# Wykres reszt
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--', linewidth=1)
plt.xlabel('Przewidywane PTS')
plt.ylabel('Reszty')
plt.title('Wykres reszt')
plt.grid(True)
plt.show()

# Histogram reszt
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.xlabel('Reszty')
plt.ylabel('Częstość')
plt.title('Histogram reszt')
plt.grid(True)
plt.show()

# Wykres standaryzowanych reszt
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, standardized_residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--', linewidth=1)
plt.xlabel('Przewidywane PTS')
plt.ylabel('Standaryzowane Reszty')
plt.title('Wykres Standaryzowanych Reszt')
plt.grid(True)
plt.show()

# QQ Plot
plt.figure(figsize=(10, 6))
sm.qqplot(standardized_residuals, line ='45')
plt.title('QQ Plot Standaryzowanych Reszt')
plt.grid(True)
plt.show()



In [ ]:
import statsmodels.api as sm
import statsmodels.api as sm
import numpy as np

# Wyznaczanie statystyk DFFITS i DFBETAS
influence = model.get_influence()
dffits = influence.dffits[0]
dfbetas = influence.dfbetas
cook_dist = influence.cooks_distance[0]

len = df.shape[0]

# Identify influential observations based on Cook's distance
df['influential_cook'] = np.where(cook_dist > 4 / len, 1, 0)

# Identify influentialobservations based on DFFITS
df['influential_dffits'] = np.where(np.abs(dffits) > 2 * np.sqrt(len * model.df_model / model.df_resid), 1, 0)

'''# Identify influential observations based on DFBETAS for a specific coefficient
coefficient_index = 0  # Index of the coefficient to analyze
df['influential_dfbetas'] = np.where(np.abs(dfbetas[:, coefficient_index]) > 2 / np.sqrt(len), 1, 0)'''

# Oblicz standaryzowane reszty
standardized_residuals = model.get_influence().resid_studentized_internal

# Dodaj standaryzowane reszty do DataFrame
df['standardized_residuals'] = standardized_residuals

# Dodaj kolumnę z wartością 1 jeśli standardized_residuals < 2, inaczej 0
df['residuals_indicator'] = np.where(np.abs(df['standardized_residuals']) > 2, 1, 0)

inf = ['residuals_indicator', 'influential_cook', 'influential_dffits']

df[inf] = df[inf].apply(pd.to_numeric, errors='coerce')

In [ ]:
#Usunięcie wpływowych
df = df[df['residuals_indicator'] == 0]
df = df[df['influential_dffits'] == 0]
df = df[df['influential_cook'] == 0]

In [ ]:
# Predykcja wartości na podstawie modelu
y_pred = model.predict(X)

# Narysowanie rzeczywistych vs przewidywanych wartości punktów
plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', lw=2)  # Linia idealna
plt.xlabel('Rzeczywiste PTS')
plt.ylabel('Przewidywane PTS')
plt.title('Rzeczywiste vs Przewidywane PTS')
plt.grid(True)

plt.show()

In [ ]:
#Dzrewo

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# Wybieranie cech i etykiet
X = df.loc[:, ['Age', 'G', 'eFG%', 'FT%', 'TRBPG', 'ASTPG', 'STLPG', 'BLKPG', 'TOVPG', 'PFPG']]
#X = df.drop(columns=['PTS', 'Rk','Player', 'Pos', 'Tm',  'G' , 'GS'  ,  'MP'  , 'FG'  , 'FGA' , '3P' , '3PA' ,   '2P' , '2PA' ,   'FT', 'FTA' ,  'PTS', 'PPG', 'MPG'])
y = df['Pos']  # Etykiety (kolumna "Pos")

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)

# Budowanie modelu drzewa decyzyjnego
dt_classifier = DecisionTreeClassifier(random_state=23)
dt_classifier.fit(X_train, y_train)

# Predykcja na zbiorze testowym
y_pred = dt_classifier.predict(X_test)

# Ewaluacja modelu
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Narysowanie drzewa decyzyjnego
plt.figure(figsize=(20, 10))  # Ustawienie rozmiaru wykresu
plot_tree(dt_classifier, filled=True, feature_names=X.columns, class_names=y.unique())
plt.show()